In [11]:
%load_ext nb_black

import implicit
import scipy
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
import catboost
import wandb
import os
import typing
from torch.utils.data import Dataset, DataLoader
import machine_learning
import shap
import matplotlib.pyplot as plt
import contextlib
from tqdm.auto import tqdm
import gradio as gr
import sys
import time
import requests
import asyncio
from enum import Enum
import machine_learning

from machine_learning.recommending.app import build_app
from machine_learning.recommending.utils import fetch_artifact, load_path_from_artifact
from machine_learning.recommending.models.slim import SLIM
from machine_learning.recommending.models.baseline import SVDRecommender
from my_tools.utils import torch_sparse_slice

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [16]:
import importlib

importlib.reload(machine_learning.recommending.app)

Trying to unpickle estimator TruncatedSVD from version 1.1.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.


<module 'machine_learning.recommending.app' from '/Users/dimakoshman/YSDA/machine_learning/recommending/app.py'>

<IPython.core.display.Javascript object>

In [17]:
from machine_learning.recommending.app import app

<IPython.core.display.Javascript object>

In [22]:
torch.save(state_dict, "svd.pt")

<IPython.core.display.Javascript object>

In [51]:
(1, 2) > (1, 5)

False

<IPython.core.display.Javascript object>

In [26]:
set([1]) == set([2])

False

<IPython.core.display.Javascript object>

In [24]:
model = None
app.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x29674c370>, 'http://127.0.0.1:7862/', None)

<IPython.core.display.Javascript object>

In [12]:
entity = "dkoshman"
project = "Recommending"
ml = MovieLens("../local/ml-100k")

artifact = fetch_artifact(entity=entity, project=project, artifact_name="slim")
checkpoint_path = load_path_from_artifact(artifact, path_inside_artifact="checkpoint")
checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))
state_dict = {
    k.split(".")[1]: v
    for k, v in checkpoint["state_dict"].items()
    if k.startswith("model.")
}
slim = SLIM()
slim.load_state_dict(state_dict)
slim = slim.eval()

NameError: name 'MovieLens' is not defined

<IPython.core.display.Javascript object>

In [19]:
wandb.finish()
wandb.init(job_type="app", project="Recommending", entity="dkoshman")
app.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x161235150>, 'http://127.0.0.1:7862/', None)

<IPython.core.display.Javascript object>

In [20]:
app.close()

Closing server running on port: 7862


<IPython.core.display.Javascript object>

In [3]:
# No need for this, just recommend
# n_background_samples_for_shap=slim.n_users

# background_samples_for_shap = slim.to_scipy_coo(slim.explicit).tocsr()[
#     np.random.choice(
#         np.arange(slim.n_users),
#         replace=False,
#         size=min(n_background_samples_for_shap, slim.n_users),
#     )
# ]

# shaps = np.zeros(slim.n_items)
# explicit = slim.to_scipy_coo(slim.explicit).tocsr()
# means = np.nan_to_num((explicit.sum(0) / (explicit > 0).sum(0)).A).squeeze(0)

# for item_id, mean in tqdm(enumerate(means[:100])):
#     item_weight = torch_sparse_slice(slim.sparse_weight, col_ids=[item_id])
#     item_weight = item_weight.to_dense().numpy().squeeze(1)
#     coefficient = slim.item_bias[item_id].item()
#     explainer = shap.explainers.Linear(
#         model=(item_weight, coefficient),
#         masker=background_samples_for_shap,
#     )
#     shap_values = explainer(background_samples_for_shap)
#     shaps += shap_values.abs.sum(0).values*mean


# items_by_importance = (-shaps).argsort()

<IPython.core.display.Javascript object>

In [1]:
wandb.define_metric()

NameError: name 'wandb' is not defined

In [249]:
from machine_learning.recommending.models import SLIM, SVDRecommender
from machine_learning.recommending.movielens.data import MovieLens25m


class Feedback(Enum):
    NONE = "Haven't seen it"
    ONE, TWO, THREE, FOUR, FIVE = map(str, range(1, 6))

    @property
    def int(self):
        return {v: i for i, v in enumerate(Feedback)}[self]


class MovieMarkdownGenerator:
    def __init__(
        self,
        links_dataframe,
        movies_dataframe,
        tmdb_api_token="0369096d5891e7959f8e5ae35bae71bc",
    ):
        self.links = links_dataframe
        self.movies_dataframe = movies_dataframe
        for dataframe in [self.links, self.movies_dataframe]:
            dataframe.set_index("movieId", inplace=True)
            match dataframe.index.min():
                case 0:
                    pass
                case 1:
                    dataframe.index -= 1
                case _:
                    raise ValueError("Index of dataframe should start with 0 or 1.")
        self.tmdb_api_token = tmdb_api_token

    def imdb_id(self, movie_id) -> str:
        imdb_id = self.links["imdbId"].loc[movie_id]
        return f"{imdb_id:07d}"

    def imdb_url(self, movie_id):
        return f"https://www.imdb.com/title/tt{self.imdb_id(movie_id)}"

    def tmdb_request_url(self, imdb_id):
        return (
            f"https://api.themoviedb.org/3/find/tt{imdb_id}"
            f"?api_key={self.tmdb_api_token}&external_source=imdb_id"
        )

    @staticmethod
    def poster_url_from_tmdb_poster_path(poster_path, size="w200"):
        return f"https://image.tmdb.org/t/p/{size}{poster_path}"

    def __call__(self, movie_id):
        imdb_id = self.imdb_id(movie_id)
        tmdb_request_url = self.tmdb_request_url(imdb_id)
        request = requests.get(tmdb_request_url)
        if not request.ok:
            wandb.log(
                dict(
                    failed_request=dict(
                        reason=request.reason,
                        status_code=request.status_code,
                        text=request.text,
                    )
                )
            )
            markdown = ""
            if self.movies_dataframe is not None:
                markdown = self.movies_dataframe.loc[movie_id]["movie title"]
            return markdown, ""

        movie_results = request.json()["movie_results"][0]
        poster_url = self.poster_url_from_tmdb_poster_path(
            poster_path=movie_results["poster_path"]
        )
        title = movie_results["title"]
        imdb_url = self.imdb_url(movie_id)
        markdown = (
            f"""[{title}]({imdb_url})""" + self.movies_dataframe.loc[movie_id]["title"]
        )
        return markdown, poster_url


class AsyncRecommender:
    def __init__(self, recommender: "RecommenderModuleBase"):
        self.recommender = recommender
        self.user_explicit = pd.Series(
            data=np.full(self.recommender.n_items, fill_value=np.nan)
        )
        self.movie_id = self.recommend()
        self.task = asyncio.create_task(self.async_recommend())

    def recommend(self):
        user_explicit = scipy.sparse.coo_matrix(
            self.user_explicit.fillna(0).values.reshape(1, -1)
        )
        user_recommendations = self.recommender.online_recommend(
            user_explicit, n_recommendations=self.recommender.n_items
        )[0].numpy()
        new_items_mask = self.user_explicit.isna().values
        new_items_positions = new_items_mask[user_recommendations].nonzero()[0]
        movie_id = user_recommendations[new_items_positions[0]]
        return movie_id

    async def async_recommend(self):
        self.user_explicit.loc[self.movie_id] = 0
        return self.recommend()

    async def next_recommendation(self, rating_of_previous_movie: int):
        wandb.log(
            dict(
                movie_id=self.movie_id,
                seen=(r := rating_of_previous_movie) > 0,
                rating=r if r > 0 else None,
            )
        )
        self.user_explicit.loc[self.movie_id] = rating_of_previous_movie
        self.movie_id = await self.task
        self.task = asyncio.create_task(self.async_recommend())
        return self.movie_id

    def __del__(self):
        wandb.log(
            dict(feedback=wandb.Table(dataframe=self.user_explicit.dropna().to_frame()))
        )


checkpoint_path = "/Users/dimakoshman/Downloads/epoch=0-step=2.ckpt"
movie_markdown_generator = MovieMarkdownGenerator(
    links_dataframe=MovieLens25m("/Users/dimakoshman/Downloads/ml-25m")["links"],
    movies_dataframe=MovieLens25m("/Users/dimakoshman/Downloads/ml-25m")["movies"],
)

checkpoint = torch.load(checkpoint_path)
state_dict = {k.removeprefix("model."): v for k, v in checkpoint["state_dict"].items()}

# model = SLIM()
model = SVDRecommender()
model.load_state_dict(state_dict)
model.eval()

Trying to unpickle estimator TruncatedSVD from version 1.1.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.


SVDRecommender()

<IPython.core.display.Javascript object>

In [252]:
with gr.Blocks() as app:
    async_recommender = AsyncRecommender(recommender=model)
    gr.Markdown("Movie recommender")
    with gr.Row():
        first_movie_markdown, first_image_url = movie_markdown_generator(
            async_recommender.movie_id
        )
        image = gr.Image(first_image_url, interactive=False)
        image.style(width=200)
        with gr.Column(scale=4):
            markdown = gr.Markdown(first_movie_markdown)

            def make_button(feedback):
                async def wrapper():
                    movie_id = await async_recommender.next_recommendation(feedback.int)
                    movie_markdown, image_url = movie_markdown_generator(movie_id)
                    return movie_markdown, image_url

                button = gr.Button(feedback.value)
                button.click(
                    fn=wrapper,
                    inputs=[],
                    outputs=[markdown, image],
                )
                return button

            none_button = make_button(Feedback.NONE)
            with gr.Row():
                buttons = [make_button(feedback) for feedback in list(Feedback)[1:]]


wandb.finish()
wandb.init(job_type="app", project="Recommending", entity="dkoshman")
app.launch()

Exception ignored in: <function AsyncRecommender.__del__ at 0x3b2559510>
Traceback (most recent call last):
  File "/var/folders/5w/gytwm8sd6vldh9ym3dvj_x_m0000gn/T/ipykernel_94050/3082034799.py", line 121, in __del__
  File "/Users/dimakoshman/Library/Caches/pypoetry/virtualenvs/ysda-adIGuw6j-py3.10/lib/python3.10/site-packages/wandb/sdk/lib/preinit.py", line 36, in preinit_wrapper
    raise wandb.Error(f"You must call wandb.init() before {name}()")
wandb.errors.Error: You must call wandb.init() before wandb.log()


Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Traceback (most recent call last):
  File "/Users/dimakoshman/Library/Caches/pypoetry/virtualenvs/ysda-adIGuw6j-py3.10/lib/python3.10/site-packages/gradio/routes.py", line 283, in run_predict
    output = await app.blocks.process_api(
  File "/Users/dimakoshman/Library/Caches/pypoetry/virtualenvs/ysda-adIGuw6j-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 892, in process_api
    result = await self.call_function(fn_index, inputs, iterator)
  File "/Users/dimakoshman/Library/Caches/pypoetry/virtualenvs/ysda-adIGuw6j-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 731, in call_function
    prediction = await block_fn.fn(*processed_input)
  File "/var/folders/5w/gytwm8sd6vldh9ym3dvj_x_m0000gn/T/ipykernel_94050/242821701.py", line 15, in wrapper
    movie_id = await async_recommender.next_recommendation(feedback.int)
  File "/var/folders/5w/gytwm8sd6vldh9ym3dvj_x_m0000gn/T/ipykernel_94050/3082034799.py", line 108, in next_recommendation
    wandb.log(
  File "/Use

In [251]:
app.close()
wandb.finish()

Closing server running on port: 7873


movie_id,▁▁▂▂▆▄█▃▃▂▃
rating,▅██▅▁▁▅██▅█
seen,▁▁▁▁▁▁▁▁▁▁▁
movie_id,317
rating,5
seen,True


<IPython.core.display.Javascript object>